In [1]:

import json
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F

connection_parameters = json.load(open('../connection.json'))
session = Session.builder.configs(connection_parameters).create()



### 1. Build docker image and push the image to image registry

- Run the below code from terminal or execute the next cell by updating the connection details

<!-- Run below commands from a terminal. Update the ORGNAME-ACCTNAME with your account info and alsp update the IMAGE_REPOSITORY, DATABASE , SCHEMA NAME and username -->

```
cd text2sql

docker build --no-cache --platform linux/amd64 -t ORGNAME-ACCTNAME.registry.snowflakecomputing.com/DATABASE/SCHEMA/IMAGE_REPOSITORY/audiollm:latest . 

docker login ORGNAME-ACCTNAME.registry.snowflakecomputing.com -u <username>

docker push ORGNAME-ACCTNAME.registry.snowflakecomputing.com/DATABASE/SCHEMA/IMAGE_REPOSITORY/audiollm:latest
```

In [29]:
%%bash
# Run below commands by updating the ORGNAME-ACCTNAME with your account info .
# Also update the IMAGE_REPOSITORY, DATABASE, SCHEMA NAME and username

# You can execute the same commands from terminal as well

cd text2sql

docker build --no-cache --platform linux/amd64 -t ORGNAME-ACCTNAME.registry.snowflakecomputing.com/DATABASE/SCHEMA/IMAGE_REPOSITORY/audiollm:latest . 

docker login ORGNAME-ACCTNAME.registry.snowflakecomputing.com -u <username>

docker push ORGNAME-ACCTNAME.registry.snowflakecomputing.com/DATABASE/SCHEMA/IMAGE_REPOSITORY/audiollm:latest


### 2. Create the Compute Pool and BIND SERVICE ENDPOINT

Run the below command in Snowsight as <b>AccountAdmin</b> . Replace role SPCS_PSE_ROLE with your own role name if you have created with different name.

``` sql
USE ROLE ACCOUNTADMIN;

CREATE COMPUTE POOL PR_GPU_7
  MIN_NODES = 1
  MAX_NODES = 1
  INSTANCE_FAMILY = GPU_NV_M
  AUTO_RESUME = FALSE
  INITIALLY_SUSPENDED = FALSE
    COMMENT = 'For text2sql' ;

GRANT USAGE, MONITOR ON COMPUTE POOL PR_GPU_7 TO ROLE SPCS_PSE_ROLE;

-- Execute the below command if you have not used audio2text container
GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO ROLE SPCS_PSE_ROLE ;

-- Below network rule and External Access INtegration is used to download the whisper mode.

-- You need to execute the below two commands only once for all the SPC services that you will be creating. If you have already created the below rule and EAI, ignore execute the below two commands.

 CREATE NETWORK RULE allow_all_rule
    TYPE = 'HOST_PORT'
    MODE= 'EGRESS'
    VALUE_LIST = ('0.0.0.0:443','0.0.0.0:80');

CREATE EXTERNAL ACCESS INTEGRATION allow_all_eai
  ALLOWED_NETWORK_RULES = (allow_all_rule)
  ENABLED = true

GRANT USAGE ON INTEGRATION allow_all_eai TO ROLE SPCS_PSE_ROLE;

```

###  3. Creating Text2SQL SPC Service

Update th YAML to change the image value before executing the below put command

image: ORGNAME-ACCTNAME.registry.snowflakecomputing.com/pr_llmdemo/public/image_repo/audiollm:latest


In [4]:

session.file.put("./llm-text2sql.yaml", "@specs",auto_compress=False)

[PutResult(source='llm-text2sql.yaml', target='llm-text2sql.yaml.gz', source_size=652, target_size=358, source_compression='NONE', target_compression='GZIP', status='UPLOADED', message='')]

In [ ]:
# Create the service. EXTERNAL_ACCESS_INTEGRATIONS(ALLOW_ALL_EAI) is created in audio2text/audio2text_setup_code.ipynb
session.sql('''
create service llama_text2sql_svc
in compute pool PR_GPU_7
from @specs
spec='llm-text2sql.yaml'
EXTERNAL_ACCESS_INTEGRATIONS = (ALLOW_ALL_EAI)
            ''').collect()

In [17]:
#  Check the status of service
import ast
res=session.sql(''' 
SELECT SYSTEM$GET_SERVICE_STATUS('llama_text2sql_svc',1)
''').collect()[0][0]
ast.literal_eval(res)[0]

{'status': 'SUSPENDED',
 'message': 'Suspended',
 'containerName': 'text2sql-container',
 'instanceId': '0',
 'serviceName': 'LLAMA_TEXT2SQL_SVC',
 'image': 'sfseeurope-us-west-ccarrero-452.registry.snowflakecomputing.com/pr_llmdemo/public/images/audiollm:latest',
 'restartCount': 0,
 'startTime': ''}

### 4. Accessing Jupyter Lab Endpoints
Wait for the status of the service to be Running before moving to next step. 

Run the below query to get the api endpoint for the <b> jupyter lab </b>. Get the <b>ingress_url </b>(this endpoint will launch jupyter notebook running Snowpark Containers)  from the below query output.

Ouput that you would get after running the below query.

Row(name='llm-audio-ep', port=8888, protocol='TCP', ingress_enabled='true', ingress_url='test123-us-west-ccarrero-452.snowflakecomputing.app')

In [27]:
session.sql('''show endpoints in service llama_text2sql_svc;
            ''').collect()[0]

# Row(name='llm-audio-ep', port=8888, protocol='TCP', ingress_enabled='true', ingress_url='test123-us-west-ccarrero-452.snowflakecomputing.app')

Row(name='llm-audio-ep', port=8888, protocol='TCP', ingress_enabled='true', ingress_url='gmtqocnn-sfseeurope-us-west-ccarrero-452.snowflakecomputing.app')

### 5. Perform the following action on the Jupyter lab which is accessible from the endpoint

<b> 1. Open the terminal in the JupyterLab(which is running in Snowpark Containers) and run the below shell script  </b>

- sh [./download_model.sh](https://github.com/Snowflake-Labs/sfguide-call-centre-analytics-with-snowflake-cortex-and-spcs/blob/main/text2sql/download_model.sh)

This file will download the Numberstation model from hugging face and downloads to the stage which is mounted

<b> 2. Run the [FineTuneModel.ipynb](https://github.com/Snowflake-Labs/sfguide-call-centre-analytics-with-snowflake-cortex-and-spcs/blob/main/text2sql/FineTuneModel.ipynb)  </b>

- Execute each cell by cell where we are fine tuning the model as well with our own dataset.

- After executing the last cell you will be running a fast api service which will expose the Number station LLM as a api endpoint.

<b> PS </b>

You need to execute step 1 and 2 only once and for next time when you start the service run the below command on the terminal which you can open from the jupyter lab:

 <b> sh [RunFastAPI.sh](https://github.com/Snowflake-Labs/sfguide-call-centre-analytics-with-snowflake-cortex-and-spcs/blob/main/text2sql/RunFastAPI.sh) </b>



### 6. Execute the below commands to create the service function

In [ ]:
session.sql('''CREATE OR REPLACE FUNCTION text2sql(text TEXT)
RETURNS VARIANT
SERVICE=llama_text2sql_svc
ENDPOINT=api
AS '/text2sql
            ''').collect()


In [ ]:
#Testing the function
session.sql('''
select text2sql('What is the distinct purpose of the calls in the last month?')::string;
''').collect()